## This notebook is to ingest the interface listing & IMAP into cloudant database

**Remember:**   
the mapping /result in docker image is mapped to local folder ~/Documents/GitHub/jupyter/result  
the mapping /input in docker image is mapped to local folder ~/Documents/GitHub/jupyter/input


link to the cloudanta database is here:   
https://d5e8ab56-62ce-4345-b1f2-33e670691507-bluemix.cloudant.com/dashboard.html

The following code is to ingest the interface listing file into cloudanta database
the interface listing has to be put in the folder /Documents/GitHub/jupyter/input  

The original location of the interface listing is in the below link:  
https://apps.na.collabserv.com/communities/service/html/communityview?communityUuid=84d3cd10-4160-48e1-abb6-f65af679ee28#fullpageWidgetId=Wabd63ea0f599_4dc3_aed5_9ffce68576e0&file=0db2badd-03b6-456b-b3c5-ab06593a55ca

In [ ]:
import sys
sys.path.append('/odm_modules')
from common_func import cloudant_conn
import pandas as pd
pd.options.display.max_rows = 5000


In [ ]:
!ls /input

### set up the environment in the beginning of notebook

In [ ]:
#environ_suffix = '_test'
environ_suffix = ''

In [ ]:
cloudant_db_name = 'interface{}'.format(environ_suffix) # the cloudant database name to be ingested
print(cloudant_db_name)
filename = 'interface.xlsx'  # the file name in the folder input
df = pd.read_excel('/input/{}'.format(filename), sheet_name = 'active_interfaces')
res  = cloudant_conn.df_2_cloudant(df, cloudant_db_name , mode = 'REPLACE')
res

## Ingest EAL information for interfaces based on IMAP  

The link to the EAL website is: 
https://ibm.decisionfocus.com/?originalUrl=%2Fworkspace%2F6473b7e703d34e664ad00131ee002e68%2Fdashboard%2F6473b7e703d34e664ad00131ee002e68%2F6473b7e703d34e664ad00131ee00b0ad  

Go to the website and download the report, select **Applications-> EAL reports -> My report** 
then choose the data items we need in our application. 

 `mv ~/Downloads/My\ Report.xlsx ~/Documents/GitHub/jupyter/input/`
 

In [ ]:
cloudant_db_name = 'imap{}'.format(environ_suffix) # the cloudant database name to be ingested
print(cloudant_db_name)
filename = 'My Report.xlsx'  # the file name in the folder input
df = pd.read_excel('/input/{}'.format(filename)).fillna('')
df = df.loc[df.State.isin(['In Production', 'In Development']) ]
#df = df.loc[df['IMAP Number'] != '-']
print(df.shape)
res  = cloudant_conn.df_2_cloudant(df, cloudant_db_name , mode = 'REPLACE', keys= ['IMAP Number'])
res

In [ ]:
df = df.loc[~df['IMAP Number'].isin( ['-', ''])]
df.shape

In [ ]:
cloudant_conn.cloudant_client.connect()

In [ ]:
my_database = cloudant_conn.cloudant_client['imap{}'.format(environ_suffix)]

In [ ]:
cloudant_conn.cloudant_client.get('interface{}'.format(environ_suffix), False)

In [ ]:
df_temp = list(my_database)

In [ ]:
df_temp = pd.DataFrame(df_temp)

In [ ]:
df_temp[df_temp['Short Name'].str.contains('HRIW')].T

In [ ]:
df_temp[['a', 'b']] = df_temp._id.str.split('-', n = 1, expand = True)
df_temp.b.apply(lambda x: len(x)).value_counts()

#df_temp['a'] = new[0]
#df_temp['b'] = new[1]

In [ ]:
df_temp.loc[df_temp._id.str.contains('-')]._id

In [ ]:
import re

In [ ]:
list(map(lambda x : print('"{}",'.format(x) ), df_temp.columns))